<a href="https://colab.research.google.com/github/FORZpewpew/CV_project/blob/main/Env_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Ванина залупа
def obtain_visible_cards_from_screenshot(): # TODO: change image_path to an image from feed

    # NOTE FOR FORZ: Эта хтонь пусть и возвращает те же типы в том же формате, но только ВИДИМЫЕ карты и их места, про скрытые/несуществующие не возвращаю ничего, гл

    baseline_x, baseline_y = 727, 291
    offset_x, offset_y, rso = 67, 24, 40


    def match_templates(image, templates, threshold=0.9):
        """
        Matches templates against the input image and returns a list of match locations.

        Parameters:
        - image (ndarray): The input grayscale image.
        - templates (dict): A dictionary of templates, where keys are labels (e.g., rank or suit names) and values are the template images.
        - threshold (float): The threshold for considering a match.

        Returns:
        - list: A list of matches as (label, x, y), where (x, y) is the top-left corner of the match.
        """
        matches = []
        for label, template in templates.items():
            result = cv2.matchTemplate(image, template, cv2.TM_CCOEFF_NORMED)
            loc = np.where(result >= threshold)
            for pt in zip(*loc[::-1]):
                matches.append((label, pt[0], pt[1], result[pt[1], pt[0]]))
        return matches


    # Paths
    image_path = 'screenshot_1.png'  # Input screenshot
    ranks_path = './icons/ranks/'  # Path to rank templates (e.g., 1.png, 2.png, ..., 13.png)
    suits_path = './icons/suits/'  # Path to suit templates (e.g., 0.png for ♥, 1.png for ♦, etc.)

    # Load input image and convert to grayscale
    image = cv2.imread(image_path)
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)


    # Load rank and suit templates
    rank_templates = {f.split('.')[0]: cv2.imread(os.path.join(ranks_path, f), cv2.IMREAD_GRAYSCALE)
                        for f in os.listdir(ranks_path)}
    suit_templates = {f.split('.')[0]: cv2.imread(os.path.join(suits_path, f))
                        for f in os.listdir(suits_path)}


    # Perform template matching
    rank_matches = match_templates(gray_image, rank_templates)
    suit_matches = match_templates(image, suit_templates)


    def determine_card_locations(rank_matches, suit_matches):
        """
        Determines card locations based on rank and suit matches and their corresponding regions.

        Parameters:
        - rank_matches (list): List of rank matches as (rank_label, x, y).
        - suit_matches (list): List of suit matches as (suit_label, x, y).

        Returns:
        - dictionary {"tableau": np_field, "foundation": np_foundation, "top_card": ret_draw}
        """
        field_cards = [[[0, 0, 0.0, 0.0] for j in range(19)] for i in range(7)]
        draw_card = [0, 0, 0.0, 0.0]
        foundation_cards = [[0, 0, 0.0, 0.0] for i in range(4)]

        ret_field = []
        ret_foundation = []
        ret_draw = Card(value=0, suit=0, visible=False)

        for rank_label, rank_x, rank_y, sim in rank_matches:
            rank_col = (rank_x - baseline_x) // offset_x
            rank_row = (rank_y - baseline_y) // offset_y

            if (rank_y < baseline_y):
                if (rank_col < 4):
                    foundation_cards[rank_col][0] = rank_label
                    foundation_cards[rank_col][2] = sim
                elif (rank_col == 5):
                    draw_card[0] = rank_label
                    draw_card[2] = sim
                continue

            if (sim > field_cards[rank_col][rank_row][2]):
                field_cards[rank_col][rank_row][0] = rank_label
                field_cards[rank_col][rank_row][2] = sim

        for suit_label, suit_x, suit_y, sim in suit_matches:
            suit_col = (suit_x - baseline_x) // offset_x
            suit_row = (suit_y - baseline_y) // offset_y

            if (suit_y < baseline_y):
                if (suit_col < 4):
                    foundation_cards[suit_col][1] = suit_label
                    foundation_cards[suit_col][3] = sim
                elif (suit_col == 6):
                    draw_card[1] = suit_label
                    draw_card[3] = sim
                continue

            if (sim > field_cards[suit_col][suit_row][3]):
                field_cards[suit_col][suit_row][1] = suit_label
                field_cards[suit_col][suit_row][3] = sim

        for i, row in enumerate(field_cards):
            for j, card in enumerate(row):
                if (card[0] != 0 and card[0] != 14):
                    ret_field.append((i, j, Card(
                        value=card[0],
                        suit=card[1],
                        visible=True)))

        for i, card in enumerate(foundation_cards):
            if (card[0] != 0 and card[0] != 14):
                    ret_foundation.append((i, Card(
                        value=card[0],
                        suit=card[1],
                        visible=True)))

        if (draw_card[0] != 14):
            if (draw_card[0] == 0):
                ret_draw = Card(
                value=0,
                suit=0,
                visible=False)
            else:
                ret_draw = Card(
                    value=draw_card[0],
                    suit=draw_card[1],
                    visible=True)
        np_field = np.array(ret_field)
        np_foundation = np.array(ret_foundation)
        np_draw = np.array(ret_draw)
        all_current_cards_ret = {
            "tableau": np_field,
            "foundation": np_foundation,
            "top_card": ret_draw
        }
        return all_current_cards_ret


    # Determine card locations
    all_current_cards = determine_card_locations(rank_matches, suit_matches)


    # Output results
    print("Detected Cards:")
    print(all_current_cards, sep='\n')

    return all_current_cards



In [1]:
%pip install gym rich gymnasium
%pip install stable_baselines3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 958.1/958.1 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 4.1 MB/s eta 0:00:00


In [2]:
class Card:
    def __init__(self, value, suit, visible=False, bonus=False):
        """
        Initialize a card with given value, suit, visibility, and bonus status.

        Parameters:
        - value (int): The rank of the card (1 to 13, where 1 = Ace and 13 = King).
        - suit (int): The suit of the card (0 to 3).
        - visible (bool): Whether the card is face-up.
        - bonus (bool): Whether this card has been given a bonus for moving to the foundation.
        """
        self.value = value  # 1 to 13
        self.suit = suit  # 0 to 3
        # 0 - ♥ (hearts) (red)
        # 1 - ♦ (diamonds) (red)
        # 2 - ♣ (clubs) (black)
        # 3 - ♠ (spades) (black)
        # for observation:
        # 4 - unexisting suit for flatten()
        # 5 - invisible suit

        self.visible = visible  # Face-up or face-down
        self.bonus = bonus  # Bonus flag to prevent duplicate rewards

    def __repr__(self):
        visibility = "Visible" if self.visible else "Hidden"
        return f"Card(value={self.value}, suit={self.suit}, {visibility}, bonus={self.bonus})"

    def to_array(self):
        """Convert card to a numeric representation."""
        # Use a tuple of (value, suit) where suit is encoded as a number.
        return [self.value, self.suit]


In [3]:
from stable_baselines3.common.env_checker import check_env
from stable_baselines3 import PPO, A2C, DQN
import gymnasium as gym
from gymnasium import spaces
import random
from rich.console import Console
from rich.text import Text
import numpy as np
import cv2
import os

In [4]:

# Initialize a Console object from the rich library for printing with styles
console = Console()

class SolitaireEnv(gym.Env):
    def __init__(self):
        super(SolitaireEnv, self).__init__()
        # The action space now includes three parts: action type, source column, destination column
        # Action type: 0 (Move within tableau), 1 (Draw card), 2 (Move to foundation); Source column (0-11) and card index (0-12); Destination column (0-6)
        self.action_space = spaces.MultiDiscrete([3, 12, 18, 7])

        # Define observation space with structured tableau, foundation, and draw pile
        self.observation_space = spaces.Dict({
            'tableau': spaces.MultiDiscrete([15, 6] * 7 * 18),  # 7 columns, 13 cards each, (value, suit)
            'foundation': spaces.MultiDiscrete([14, 14, 14, 14]),  # 4 foundation piles
            'top_card': spaces.MultiDiscrete([15, 6])  # Top card of the draw pile, (value, suit)
        })

        # Track revealed cards from the draw pile
        self.revealed_cards = []
        self.draw_index = 0  # Tracks current position in draw pile

        self.tableau = None
        self.foundation = None
        self.draw_pile = None
        self.draw_pile_cycles = 3
        self.done = False
        self.reward = 0
        self.colors = {
            0: "red",
            1: "red",
            2: "black",
            3: "black"
        }
        self.king_value = 13
        self.ace_value = 1
        self.invisible_value = 14
        self.unexisting_value = 0
        self.invisible_suit = 5
        self.unexisting_suit = 4
        self._reset_game_state()

    def _reset_game_state(self):
        # Initialize the deck as a list of Card objects
        deck = [Card(value, suit) for suit in range(4) for value in range(self.ace_value, self.king_value+1)]
        random.shuffle(deck)

        # Initialize tableau with some cards face-down
        self.tableau = [[deck.pop() for _ in range(i + 1)] for i in range(7)]
        for col in self.tableau:
            for card in col[:-1]:
                card.visible = False  # Face-down
            col[-1].visible = True  # Top card face-up

        # Foundation starts empty
        self.foundation = [[] for _ in range(4)]
        self.draw_pile = deck  # Remaining cards in the draw pile
        self.done = False
        self.reward = 0

    def reset(self, seed=None):
        """Resets the environment to the initial state."""
        self._reset_game_state()
        return self._get_observation(), {}


    def _get_observation(self):
        # This function will need to convert the complex Card objects to a simple format for Gym
      tableau_observation = [
          [card.to_array() if card.visible else [self.invisible_value, self.invisible_suit] for card in column]
          for column in self.tableau
      ]

      # Foundation observation (number of cards in each pile)
      foundation_observation = [len(pile) for pile in self.foundation]

      # Draw pile observation: get top card's value, suit
      if self.revealed_cards:
          top_card = self.revealed_cards[-1]
          top_card_observation = top_card.to_array()  # Use the to_array method to get (value, suit, visible)
      else:
          # If no cards in the draw pile, return default unexisting value
          top_card_observation = [self.unexisting_value, self.unexisting_suit]  # Represent empty pile as a tuple

      # Max length for tableau columns
      max_length = 18
      tableau_observation_padded = [
        stack + [[self.unexisting_value, self.unexisting_suit]] * (max_length - len(stack)) for stack in tableau_observation
      ]

      # Convert tableau to a NumPy array and flatten it
      try:
        tableau_array = np.array(tableau_observation_padded).flatten()
      except:
        print(tableau_observation_padded)

      # Final result combining all parts of the observation
      result = {
          "tableau": tableau_array,
          "foundation": np.array(foundation_observation),
          "top_card": np.array(top_card_observation)
      }
      #print(tableau_array)
      return result

    def step(self, action: list):
        # action: [int, list[int,int], int]
        # action_type: int
        # source: list[int,int] - [column, card in column]. Column numbers:
        # 0-6: tableau
        # 7-10: foundation
        # 11: draw pile
        # destination: int - column, no need  since all the cards will be moved on top of it

        action_type, source1, source2, destination = action
        current_reward = -1   # Base penalty for each action

        if action_type == 0:  # Move Card within Tableau
            valid_move_made, reward = self._move_within_tableau([source1, source2], destination)   # here reward should be negative
            current_reward += reward
            if not valid_move_made:
                #print("The move isn't valid")
                current_reward -= 10  # Extra penalty for invalid move

        elif action_type == 1:  # Draw Card from Draw Pile
            empty = self._draw_card()
            current_reward -= 70
            if empty:
              current_reward -= 100

        elif action_type == 2:  # Move Card to Foundation
            valid_move_made, reward = self._move_to_foundation(source1)
            current_reward += reward

        flipped_count = self._flip_visible_cards()
        current_reward += flipped_count * 1600

        # Check if game is won (all foundations complete)
        if all(len(foundation) == 13 for foundation in self.foundation):
            self.done = True

        self.reward += current_reward

        return self._get_observation(), current_reward, self.done, False, {}  # False - truncated field (hz zachem), {} - info field (tozhe hz zachem)

    def _draw_card(self):
        # Reveal 1 card at a time from the draw pile
        if self.draw_pile:
            card = self.draw_pile.pop()
            card.visible = True
            self.revealed_cards.append(card)
            return False
        else:
            self.draw_pile = self.revealed_cards[::-1]  # Restart the draw pile if we reach the end
            self.revealed_cards = []
            self.draw_pile_cycles-=1
            if self.draw_pile_cycles < 0:
                return True


    def _is_alternating_color(self, object1, object2):
        # Check if the objects have alternating colors
        return self.colors[object1] != self.colors[object2]


    # all return numbers after false are negative, and positive after true
    def _move_within_tableau(self, source: list[int], destination: int):
        if destination > 6 or destination < 0:
          #print("Wrong destination column")
          return False, -10
        # If the source is from the draw pile
        if source[0] == 11:
            if not self.revealed_cards:
                #print("Invalid move: No cards revealed in the draw pile")
                return False, -50 # No cards revealed in the draw pile
            # Use the last revealed card from the draw pile
            card_to_move = self.revealed_cards[-1]

            # Check if destination column is empty (only Kings can move to empty columns)
            if not self.tableau[destination]:
                if card_to_move.value == self.king_value: # King card value
                    self.tableau[destination].append(card_to_move)
                    self.revealed_cards.pop()  # Remove from revealed list
                    return True, 500
                else:
                    #print("Invalid move: Only Kings can move to an empty column")
                    return False, -60  # Only Kings can move to an empty column

            # Check if the move is valid based on the destination column's top card
            dest_card = self.tableau[destination][-1]
            if (card_to_move.value == dest_card.value - 1 and
                self._is_alternating_color(card_to_move.suit, dest_card.suit)):
                self.tableau[destination].append(card_to_move)
                self.revealed_cards.pop()  # Remove from revealed list
                return True, 500

            #print("Invalid move: Invalid move for draw pile card")
            return False, -50  # Invalid move for draw pile card

        if 7 <= source[0] <= 10:
          suit = source[0] - 7
          if self.foundation[suit] and self.tableau[destination]:
              card_to_move = self.foundation[suit][-1]
              dest_card = self.tableau[destination][-1]
              if (card_to_move.value == dest_card.value - 1 and
                  self._is_alternating_color(card_to_move.suit, dest_card.suit)):
                  self.tableau[destination].append(card_to_move)
                  self.foundation[suit].pop()  # Remove from revealed list
                  return True, 500
              return False, -50
          else:
            return False, -50

        if source[0] < 0:
            #print("Invalid move: Wrong column number")
            return False, -100
        if source[1] >= len(self.tableau[source[0]]):
            #print("Invalid move: Wrong card index number")
            return False, -100

        card_column = source[0]
        card_index = source[1]
        cards_to_move = self.tableau[card_column][card_index:]

        # Check if destination column is empty (only Kings can be moved to an empty column)
        if not self.tableau[destination]:
            if cards_to_move[0].value == self.king_value: # King card value
                # Move the sequence
                self.tableau[destination].extend(cards_to_move)
                del self.tableau[card_column][card_index:]
                return True, 500
            else:
                #print("Invalid move: Only Kings can be moved to an empty column")
                return False, -60  # Only Kings can be moved to an empty column

        # Check if the move is valid based on the destination column’s top card
        dest_card = self.tableau[destination][-1]
        if (cards_to_move[0].value == dest_card.value - 1 and
                self._is_alternating_color(cards_to_move[0].suit, dest_card.suit)):
            # Move the sequence
            self.tableau[destination].extend(cards_to_move)
            del self.tableau[card_column][card_index:]
            return True, 500

        #print("Invalid move: Invalid move within tableau")
        return False, -40  # Move was invalid


    def _move_to_foundation(self, source): # source is int, since we move the top card of source to top of foundation
        valid_reward = 1300
        # If source is the draw pile (denoted by 11), take the last revealed card
        if source == 11:
            if not self.revealed_cards:
                #print("Invalid move: No revealed cards in draw pile")
                return False, -50  # No revealed cards in draw pile
            card = self.revealed_cards[-1]
            foundation_index = card.suit # Determine foundation based on suit

            if len(self.foundation[foundation_index]) == card.value - self.ace_value:  # -ace_value because card values are from 2 to 14
                # Move the card to the foundation and remove from revealed list
                self.foundation[foundation_index].append(card)
                self.revealed_cards.pop()
                if not card.bonus:
                    card.bonus = True
                return True, valid_reward

            #print("Invalid move: Invalid move to foundation from draw pile")
            return False, -40  # Invalid move

        # Validate source column
        if source < 0 or source > 6 or not self.tableau[source]:
            #print("Invalid move: No card to move")
            return False, -60  # Invalid move, no card to move

        # Get the top card from the source column
        card = self.tableau[source][-1]
        foundation_index = card.suit  # Determine foundation pile based on suit

        # Check if the card can move to the foundation (must be in ascending order)
        if len(self.foundation[foundation_index]) == card.value - self.ace_value:     # -ace_value because card values are from 2 to 14
            # Move the card to the foundation and remove from tableau
            self.foundation[foundation_index].append(self.tableau[source].pop())
            if not card.bonus:
                card.bonus = True
            return True, valid_reward

        #print("Invalid move: Invalid move to foundation from tableau")
        return False, -40  # Move was invalid


    def _flip_visible_cards(self):
        flipped_count = 0
        for column in self.tableau:
            if column and not column[-1].visible:  # If the top card is face-down
                column[-1].visible = True  # Flip it face-up
                flipped_count += 1
        return flipped_count


    def render(self, mode='human'):
        # Foundations
        foundation_str = []
        for pile in self.foundation:
            if pile:
                card = pile[-1]
                suit = ['[bold red] ♥[/bold red]', '[bold red] ♦[/bold red]', ' ♣', ' ♠'][card.suit]
                # Apply color red for red suits (Diamonds and Hearts)
                foundation_str.append(f"| {card.value} {suit} |" if card.value != self.insivible_value else f"| A {suit} |")
            else:
                foundation_str.append("|     |")  # Empty foundation pile

        # Print foundation row
        console.print("Foundations:", "  ".join(foundation_str))

        # Tableau - display all 7 columns in a single row
        tableau_str = []
        for col in self.tableau:
            tableau_str.append(" ".join([f"┌─────┐" if card.visible else "┌─────┐" for card in col]))  # Card tops
            tableau_str.append(" ".join([f"| {card.value if card.visible else ' ?'}{' ' if card.visible and len(str(card.value)) != 2 else ''}{['[bold red] ♥[/bold red]', '[bold red] ♦[/bold red]', ' ♣', ' ♠'][card.suit] if card.visible else '  '}|" for card in col]).replace(" 0 ", " A "))  # Card values
            tableau_str.append(" ".join([f"|     |" for _ in col]))  # Empty space for spacing between cards
            tableau_str.append(" ".join([f"└─────┘" for _ in col]))  # Card bottoms

        # Print tableau columns in one row
        tableau_str = '\n'.join(tableau_str)
        console.print("Tableau:\n" + tableau_str)

        # Draw pile (remaining count, last 3 revealed, discarded count)
        draw_pile_display = f"Draw Pile: {len(self.draw_pile)} cards remaining"

        # Last 3 revealed cards (if any)
        last_three = [f"|{card.value if card.visible else ' ?'}{' ' if card.visible and len(str(card.value)) != 2 else ''}{['[bold red] ♥[/bold red]', '[bold red] ♦[/bold red]', ' ♣', ' ♠'][card.suit] if card.visible else '  '}|" for card in self.revealed_cards[-3:]]
        last_three_display = f"\nLast 3 Drawn: {' '.join(last_three)}"

        # Discarded cards (number of discarded cards)

        # Print the full draw pile row
        console.print(draw_pile_display, last_three_display)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
env = SolitaireEnv()
check_env(env, warn=True)

# Я НЕ ПОНИМАЮ ЧТО ОНО ХОЧЕТ. Я ДАЮ ЕМУ  182 ЗНАЧЕНИЯ.
# КОТОРЫЕ ПОПАРНО ОТ 0 ДО 14 И ОТ 0 ДО 3 (ха ха, оно было от 0 до 3 и 14), А ЕМУ ВСЁ РАВНО

In [61]:
print(env.observation_space.sample())

{'foundation': array([12,  2,  8,  3]), 'tableau': array([ 3,  3,  2,  3, 13,  0,  2,  3,  8,  3,  9,  4, 11,  2, 13,  0, 14,
        0,  9,  2, 10,  5, 14,  2,  7,  1,  3,  2, 11,  1,  3,  1, 11,  2,
       13,  3,  0,  0, 11,  4, 10,  4, 11,  4, 11,  5, 10,  0, 12,  4, 14,
        5, 10,  3,  0,  2,  2,  4,  3,  1, 14,  3,  5,  4,  8,  4,  0,  3,
       12,  1,  7,  4, 11,  4,  1,  3, 13,  3,  7,  4,  5,  0, 12,  0, 14,
        1,  8,  0,  2,  2,  2,  3,  7,  3, 14,  2,  8,  0, 14,  3, 13,  1,
       10,  2,  5,  5,  6,  0, 10,  3,  9,  1,  1,  1,  1,  0, 12,  3,  5,
        3,  7,  1,  7,  1,  3,  5, 10,  5,  8,  5,  1,  5,  7,  0,  2,  2,
       14,  5, 14,  1, 12,  5,  1,  2,  0,  1,  3,  5, 14,  3, 13,  3, 14,
        2, 13,  2, 13,  1,  2,  5,  6,  3,  1,  5, 13,  0,  2,  0,  3,  2,
       11,  0,  9,  0,  0,  2,  6,  4, 11,  5, 11,  4,  6,  4,  5,  4,  3,
        3, 10,  0,  1,  4, 12,  4,  8,  0,  5,  4,  9,  5, 10,  1, 14,  2,
        8,  4,  0,  4, 11,  1,  6,  0,  2,  1,  2

In [6]:
model = PPO("MultiInputPolicy", env, verbose=1).learn(400000)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
-----------------------------
| time/              |      |
|    fps             | 109  |
|    iterations      | 1    |
|    time_elapsed    | 18   |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 94          |
|    iterations           | 2           |
|    time_elapsed         | 43          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.004526401 |
|    clip_fraction        | 0.00591     |
|    clip_range           | 0.2         |
|    entropy_loss         | -8.42       |
|    explained_variance   | -2.99e-05   |
|    learning_rate        | 0.0003      |
|    loss                 | 5.84e+05    |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.00679    |
|    value_loss         

In [9]:
obs, _ = env.reset()
n_steps = 20
for step in range(n_steps):
    action, _ = model.predict(obs)
    print(f"Step {step + 1}")
    print("Action: ", action)
    obs, reward, done, truncated, info = env.step(action)
    #print("obs=", obs, "reward=", env.reward, "done=", done)
    print("reward= ", env.reward)
    env.render()
    if done:
        # Note that the VecEnv resets automatically
        # when a done signal is encountered
        print("Goal reached!", "reward=", reward)
        break

Step 1
Action:  [ 2  1 11  6]
reward=  -41


Foundations: |     |  |     |  |     |  |     |

Tableau:
┌─────┐
| 2  ♥|
|     |
└─────┘
┌─────┐ ┌─────┐
|  ?  | | 10 ♠|
|     | |     |
└─────┘ └─────┘
┌─────┐ ┌─────┐ ┌─────┐
|  ?  | |  ?  | | 12 ♠|
|     | |     | |     |
└─────┘ └─────┘ └─────┘
┌─────┐ ┌─────┐ ┌─────┐ ┌─────┐
|  ?  | |  ?  | |  ?  | | 8  ♥|
|     | |     | |     | |     |
└─────┘ └─────┘ └─────┘ └─────┘
┌─────┐ ┌─────┐ ┌─────┐ ┌─────┐ ┌─────┐
|  ?  | |  ?  | |  ?  | |  ?  | | 5  ♥|
|     | |     | |     | |     | |     |
└─────┘ └─────┘ └─────┘ └─────┘ └─────┘
┌─────┐ ┌─────┐ ┌─────┐ ┌─────┐ ┌─────┐ ┌─────┐
|  ?  | |  ?  | |  ?  | |  ?  | |  ?  | | 12 ♦|
|     | |     | |     | |     | |     | |     |
└─────┘ └─────┘ └─────┘ └─────┘ └─────┘ └─────┘
┌─────┐ ┌─────┐ ┌─────┐ ┌─────┐ ┌─────┐ ┌─────┐ ┌─────┐
|  ?  | |  ?  | |  ?  | |  ?  | |  ?  | |  ?  | | 9  ♠|
|     | |     | |     | |     | |     | |     | |     |
└─────┘ └─────┘ └─────┘ └─────┘ └─────┘ └─────┘ └─────┘

Draw Pile: 24 cards remaining 
Last 3 Drawn: |7  ♦| |6  ♦| |8  ♥|

Step 2
Action:  [ 2  1 11  6]
reward=  -82


...

Draw Pile: 24 cards remaining 
Last 3 Drawn: |7  ♦| |6  ♦| |8  ♥|

Step 20
Action:  [ 2  1 11  6]
reward=  -820


Foundations: |     |  |     |  |     |  |     |

Tableau:
┌─────┐
| 2  ♥|
|     |
└─────┘
┌─────┐ ┌─────┐
|  ?  | | 10 ♠|
|     | |     |
└─────┘ └─────┘
┌─────┐ ┌─────┐ ┌─────┐
|  ?  | |  ?  | | 12 ♠|
|     | |     | |     |
└─────┘ └─────┘ └─────┘
┌─────┐ ┌─────┐ ┌─────┐ ┌─────┐
|  ?  | |  ?  | |  ?  | | 8  ♥|
|     | |     | |     | |     |
└─────┘ └─────┘ └─────┘ └─────┘
┌─────┐ ┌─────┐ ┌─────┐ ┌─────┐ ┌─────┐
|  ?  | |  ?  | |  ?  | |  ?  | | 5  ♥|
|     | |     | |     | |     | |     |
└─────┘ └─────┘ └─────┘ └─────┘ └─────┘
┌─────┐ ┌─────┐ ┌─────┐ ┌─────┐ ┌─────┐ ┌─────┐
|  ?  | |  ?  | |  ?  | |  ?  | |  ?  | | 12 ♦|
|     | |     | |     | |     | |     | |     |
└─────┘ └─────┘ └─────┘ └─────┘ └─────┘ └─────┘
┌─────┐ ┌─────┐ ┌─────┐ ┌─────┐ ┌─────┐ ┌─────┐ ┌─────┐
|  ?  | |  ?  | |  ?  | |  ?  | |  ?  | |  ?  | | 9  ♠|
|     | |     | |     | |     | |     | |     | |     |
└─────┘ └─────┘ └─────┘ └─────┘ └─────┘ └─────┘ └─────┘

Draw Pile: 24 cards remaining 
Last 3 Drawn: |7  ♦| |6  ♦| |8  ♥|

In [ ]:
from IPython.display import clear_output
import sys

def main():
    # Initialize the environment
    env = SolitaireEnv()
    done = False

    # Game loop
    while not done:
        # Render the current state
        clear_output(wait=True)
        print(f"\nReward: {env.reward}")
        env.render()

        # Get user input for action
        try:
            print("\nEnter your action:")
            action_type = int(input("Action Type (0: Move within tableau, 1: Draw card, 2: Move to foundation): \n"))
            if action_type == 0:  # Move within tableau
                source_col = int(input("Source Column (0-6 for tableau, 7-10 for foundation, 11 for draw pile): \n"))
                if source_col == 11:
                    source_card_idx= 0
                else:
                    source_card_idx = int(input("Source Card Index (starting from 1 for the bottom card): \n"))
                dest_col = int(input("Destination Column (0-6): \n"))
                action = [action_type, [source_col, source_card_idx], dest_col]
            elif action_type == 1:  # Draw card
                action = [action_type, [], 0]  # No source or destination needed for draw action
            elif action_type == 2:  # Move to foundation
                source_col = int(input("Source Column (0-6 for tableau, 11 for draw pile): \n"))
                action = [action_type, [source_col, 0], 0]  # No destination needed for foundation
            else:
                print("Invalid action type! Try again.")
                continue
        except ValueError:
            print("Invalid input! Please enter valid numbers.")
            continue

        # Take the action
        try:
            obs, reward, done, terminal, info = env.step(action)
            if done:
                print("Congratulations! You have completed the game!\n Your score: ", env.reward)
        except Exception as e:
            print(f"Error: {e}")
            sys.exit(1)

if __name__ == "__main__":
    main()
